### num_rows 기준 정하고 그에 해당하는 시퀀스 모두 제거하는 코드

- 코드설명
1. del_idx 컬럼의 값을 가진 csv file 불러오기 (bitcoin_prepro_v2.ipynb에서 작업)
2. num_rows 기준에 해당하는 값과 그 값이 포함된 시퀀스 제외한 최종 시퀀스 생성
3. 시퀀스 정규화(MinMaxScale) 및 tensor 형태로 변환
4. time gap이 존재하는 부분은 sequence에서 제거
5. 모든 sequence 기준(20, 40, ... 320)에 따른 pickle 파일 생성 및 저장

In [4]:
# 필요 라이브러리 import

# Pytorch
import torch
import torch.optim as optim
import torch.nn as nn

# Dataset 관련
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
import sequence as sq # 사용자 정의 함수 불러오기

# 성능 평가 관련
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import confusion_matrix
from collections import Counter

# Visualization 관련
import matplotlib.pyplot as plt
import seaborn as sns

# 하이퍼파라미터 튜닝
import optuna
from optuna.pruners import MedianPruner
from sklearn.model_selection import TimeSeriesSplit

# 운영체제 관련
import platform

'''
딥러닝 학습을 진행할 때, 가중치를 임의의 값으로 초기화하여 학습을 수행하는 데, 
실험을 동일하게 진행하기 위해서는 난수를 동일하게 생성해야 한다.
Pytorch에서 random seed를 고정하기 위해 manual_seed를 사용한다.
'''

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)  # 멀티 GPU 사용 시
# GPU에서 실행할 때, CUDNN 자동 튜너의 비결정적 행동을 방지하기 위해 이를 설정
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# 운영체제별 device 설정
os_name = platform.system()
if os_name == 'Windows':
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"이 PC는 윈도우 운영 체제입니다: {device} is available")
elif os_name == 'Darwin':
    device = torch.device("mps" if torch.backends.mps.is_available else "cpu")
    print(f"이 PC는 맥(OS X) 운영 체제입니다: {device} is available")
else:
    print(f"이 PC는 다른 운영 체제입니다: {os_name}")


이 PC는 윈도우 운영 체제입니다: cuda:0 is available


In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [6]:
# 데이터 불러오기
file_path = '../../data/' # 경로 설정
df = pd.read_csv(file_path + 'bitcoin_data_num_rows_gt_5.csv')
#df = df.iloc[:10000]
#df['returns_next10m'] = df['returns_next10m'].apply(lambda x: 0 if x <= 0 else 1) # 종속변수 이진분류화

# transaction fee를 고려한 종속변수
# transaction_fee = 0.001
# df['returns_next10m'] = df['returns_next10m'] - transaction_fee # subtract transaction fee

df['returns_next10m_binary'] = df['returns_next10m'].apply(lambda x: 0 if x <= 0 else 1) # 종속변수 이진분류화
df = df.sort_values(by='window_start', ascending=True) # 시간순 정렬

# sequence length를 기준으로 sequence 데이터 생성
seq_len = 20 # 20, 40, 80, 160, 320
#X, y = sq.create_sequence(df, seq_len=seq_len) # 사용자 정의 함수
X, y, y_for_backtest = sq.createSeqForBacktestwithTimeGapDelete(df, seq_len=seq_len)

# timegap 포함되는 sequence도 삭제. 수행 결과 데이터의 개수가 6만개 정도만 남는것으로 거의 반토막남.
#X, y, y_for_backtest = sq.createSeqForBacktestwithTimeGapDelete(df, seq_len=seq_len)

# Tensor화
X = torch.FloatTensor(X).to(device)
y = torch.FloatTensor(y).to(device)
print('Full Data Size:', X.size(), y.size())

# split (60% / 20% / 20%)
train_split = int((X.size(0)) * 0.6)
valid_split = int((X.size(0)) * 0.8)

X_train_seq = X[:train_split]
X_val_seq = X[train_split:valid_split]
X_test_seq = X[valid_split:]
y_train_seq = y[:train_split]
y_val_seq = y[train_split:valid_split]
y_test_seq = y[valid_split:]
y_test_bt = y_for_backtest[valid_split:] # for backtest

print('Train Size:', X_train_seq.size(), y_train_seq.size())
print('Valid Size:', X_val_seq.size(), y_val_seq.size())
print('Test Size:', X_test_seq.size(), y_test_seq.size())


AttributeError: module 'sequence' has no attribute 'createSeqForBacktestwithTimeGapDelete'

In [4]:
# for verification
# verification_row = df[df['window_start'] == '2022-12-16 21:05:30'][['window_start',
#                                                                     'lowest_return',
#                                                                     'highest_return',
#                                                                     'highest_possible_return',
#                                                                     'returns_next10m']]
# verification_row

#### pickle 파일 생성 및 저장

In [5]:
# Pickle 모듈 사용하여 리스트 파일로 내보내기
import os
import pickle
file_path = '../../data/'

# 폴더가 없으면 생성
if not os.path.exists(file_path):
    os.makedirs(file_path)

# 리스트 파일로 저장
with open(f'{file_path}x_train_seq_{seq_len}.pkl', 'wb') as file: # x_train
    pickle.dump(X_train_seq, file)

with open(f'{file_path}x_val_seq_{seq_len}.pkl', 'wb') as file: # x_valid
    pickle.dump(X_val_seq, file)

with open(f'{file_path}y_train_seq_{seq_len}.pkl', 'wb') as file: # y_train
    pickle.dump(y_train_seq, file)

with open(f'{file_path}x_test_seq_{seq_len}.pkl', 'wb') as file: # x_test
    pickle.dump(X_test_seq, file)

with open(f'{file_path}y_val_seq_{seq_len}.pkl', 'wb') as file: # y_valid
    pickle.dump(y_val_seq, file)

with open(f'{file_path}y_test_seq_{seq_len}.pkl', 'wb') as file: # y_test
    pickle.dump(y_test_seq, file)

with open(f'{file_path}y_test_bt_{seq_len}.pkl', 'wb') as file: # y_test_bt
    pickle.dump(y_test_bt, file)

print(f'{seq_len} sequence data export finished')

20 sequence data export finished


In [12]:
# 불러오기
with open(f'{file_path}x_train_seq_{seq_len}.pkl', 'rb') as file:
     loaded_lst= pickle.load(file)

In [ ]:
# Mac에서 돌리면 kernel 뻑남..
# tmp_seq = []

# # 가정: create_sequence 함수는 이미 정의되어 있으며, df 데이터프레임과 sequence_length 값을 입력으로 받음
# sequence_lengths = [20, 40, 80, 160, 320]

# # 리스트 컴프리헨션을 사용하여 각 sequence_length에 대한 결과의 길이 계산
# tmp_seq = [len(create_sequence(df, length)) for length in sequence_lengths]

# # 결과를 pandas DataFrame으로 변환
# result_df = pd.DataFrame({
#     'sequence_length': sequence_lengths,
#     'length': tmp_seq
# })

# result_df

In [ ]:
# data export
#result_df.to_csv(file_path+'seq_data_counts.csv', index=False)

In [5]:
import sequence

In [6]:
X, y = sequence.create_sequence(df, 20)
print(X.size(), y.size())

torch.Size([111077, 20, 77]) torch.Size([111077, 1])
